In [ ]:
print('hello')

hello


In [38]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3193640 sha256=07e41887c48ac25e383a752d22106796de30e99f8b52d2ddc44c001b981eaad0
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [39]:
import numpy as np
import pandas as pd 
import math # 불러오기 수학과 관련된 함수
import re # 정규표현식 함수 , 판별 여부
from scipy.sparse import csr_matrix # 매트릭스 해줌
import matplotlib.pyplot as plt # 시각화 함수
import seaborn as sns # 시각화 함수
from surprise import Reader, Dataset, SVD, NormalPredictor, KNNBasic # 알고리즘이 들어감
from surprise import KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise import BaselineOnly, SVDpp,NMF, SlopeOne, CoClustering # 분석툴
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import cross_validate, KFold, train_test_split # 교차 검증
import json
import mpmath
from scipy.special import logsumexp

from google.colab import drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/ml_plus/data')

In [101]:
df =  pd.read_csv('영화 & TV 프로그램 .csv', index_col= 0)

In [ ]:
df

,contentId,title,genre,content,imgUrl,contentRating,createdYear,type
0,631842,똑똑똑,"['Horror', 'Mystery', 'Thriller']",휴가를 떠난 한 가족은 별장에 무단침입한 낯선 방문자들과 대치하게 된다. ‘레너드’...,https://image.tmdb.org/t/p/w342/mCDSOfcVJfMkGU...,6.6,2023-02-01,movie
1,505642,블랙 팬서: 와칸다 포에버,"['Action', 'Adventure', 'Science Fiction']",국왕이자 블랙 팬서인 티찰라의 죽음 이후 수많은 강대국으로부터 위협을 받게 된 와칸...,https://image.tmdb.org/t/p/w342/3PCRWLeqp5y20k...,7.4,2022-11-09,movie
2,315162,장화신은 고양이: 끝내주는 모험,"['Animation', 'Adventure', 'Comedy', 'Family']",아홉 개의 목숨 중 단 하나의 목숨만 남은 장화신은 고양이. 마지막 남은 목숨을 ...,https://image.tmdb.org/t/p/w342/rKgvctIuPXyuqO...,8.4,2022-12-07,movie
3,1011679,Shark Side of the Moon,"['Action', 'Science Fiction', 'Thriller', 'Hor...",NaN,https://image.tmdb.org/t/p/w342/v5CfpzxoJDkZxj...,4.9,2022-08-12,movie
4,646389,플레인,"['Action', 'Adventure', 'Thriller']","평소와 다를 것 없던 어느 날, 기장 토렌스는 비행기 운행 중 거대한 폭풍우를 만나...",https://image.tmdb.org/t/p/w342/8XDLgSlM6hb01i...,6.9,2023-01-12,movie
...,...,...,...,...,...,...,...,...
19887,2824,The Adventures of Pete & Pete,"['Drama', 'Comedy']",NaN,https://image.tmdb.org/t/p/w342/kWg499OrdZlnW6...,7.3,1993-11-28,tv
19888,62068,게게게의 키타로,"['Animation', 'Comedy', 'Family']",NaN,https://image.tmdb.org/t/p/w342/AaX2TCBeMeV4Te...,8.0,2007-04-01,tv
19889,71180,Waterboyy the Series,"['Comedy', 'Drama']",NaN,https://image.tmdb.org/t/p/w342/cjjZO1l99kVKFa...,5.0,2017-04-09,tv
19890,89503,카카후카카 -악화된 어른들의 쉐어하우스-,"['Comedy', 'Drama']",셰어하우스를 무대로 펼쳐지는 어른들의 러브스토리,https://image.tmdb.org/t/p/w342/e39StHOcg9uihQ...,6.3,2019-04-25,tv


In [181]:
x = df.drop(['content','imgUrl','createdYear','type'],axis=1)

In [182]:
x

,contentId,title,genre,contentRating
0,631842,똑똑똑,"['Horror', 'Mystery', 'Thriller']",6.6
1,505642,블랙 팬서: 와칸다 포에버,"['Action', 'Adventure', 'Science Fiction']",7.4
2,315162,장화신은 고양이: 끝내주는 모험,"['Animation', 'Adventure', 'Comedy', 'Family']",8.4
3,1011679,Shark Side of the Moon,"['Action', 'Science Fiction', 'Thriller', 'Hor...",4.9
4,646389,플레인,"['Action', 'Adventure', 'Thriller']",6.9
...,...,...,...,...
19887,2824,The Adventures of Pete & Pete,"['Drama', 'Comedy']",7.3
19888,62068,게게게의 키타로,"['Animation', 'Comedy', 'Family']",8.0
19889,71180,Waterboyy the Series,"['Comedy', 'Drama']",5.0
19890,89503,카카후카카 -악화된 어른들의 쉐어하우스-,"['Comedy', 'Drama']",6.3


In [104]:
x['genre'] = x['genre'].str.replace("&","")

In [105]:
x['genre'] = x['genre'].str.replace("[","")

<ipython-input-105-27d891f09f11>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x['genre'] = x['genre'].str.replace("[","")


In [106]:
x['genre'] = x['genre'].str.replace("]","")

<ipython-input-106-414c9cee33d3>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x['genre'] = x['genre'].str.replace("]","")


In [107]:
x['genre'] = x['genre'].str.replace("'","")

In [108]:
x['genre'] = x['genre'].str.replace(",","")

In [109]:
x

,contentId,title,genre,contentRating
0,631842,똑똑똑,Horror Mystery Thriller,6.6
1,505642,블랙 팬서: 와칸다 포에버,Action Adventure Science Fiction,7.4
2,315162,장화신은 고양이: 끝내주는 모험,Animation Adventure Comedy Family,8.4
3,1011679,Shark Side of the Moon,Action Science Fiction Thriller Horror,4.9
4,646389,플레인,Action Adventure Thriller,6.9
...,...,...,...,...
19887,2824,The Adventures of Pete & Pete,Drama Comedy,7.3
19888,62068,게게게의 키타로,Animation Comedy Family,8.0
19889,71180,Waterboyy the Series,Comedy Drama,5.0
19890,89503,카카후카카 -악화된 어른들의 쉐어하우스-,Comedy Drama,6.3


In [110]:
x = x.dropna(axis=0)

In [111]:
x = x.reset_index()

In [112]:
x = x.drop('index',axis=1)

In [113]:
x = x.drop_duplicates(['title']).reset_index().drop('index',axis=1)

In [114]:
x

,contentId,title,genre,contentRating
0,631842,똑똑똑,Horror Mystery Thriller,6.6
1,505642,블랙 팬서: 와칸다 포에버,Action Adventure Science Fiction,7.4
2,315162,장화신은 고양이: 끝내주는 모험,Animation Adventure Comedy Family,8.4
3,1011679,Shark Side of the Moon,Action Science Fiction Thriller Horror,4.9
4,646389,플레인,Action Adventure Thriller,6.9
...,...,...,...,...
19085,75816,Wilder,Crime,7.4
19086,2824,The Adventures of Pete & Pete,Drama Comedy,7.3
19087,71180,Waterboyy the Series,Comedy Drama,5.0
19088,89503,카카후카카 -악화된 어른들의 쉐어하우스-,Comedy Drama,6.3


In [ ]:
# 장르 백터화 
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
counter_vector = CountVectorizer(ngram_range=(1,3))

In [ ]:
c_vector_genre = counter_vector.fit_transform(x['genre'])

In [ ]:
c_vector_genre.shape

(19090, 2114)

In [ ]:
c_vector_genre

<19090x2114 sparse matrix of type '<class 'numpy.int64'>'
	with 100864 stored elements in Compressed Sparse Row format>

In [ ]:
# 유사도 값 추출 ( 코사인 유사도 )
from sklearn.metrics.pairwise import cosine_similarity

similarity_genre = cosine_similarity(c_vector_genre, c_vector_genre).argsort()[:,::-1]
print(similarity_genre)

[[    0  1120  2701 ... 12503 12502  9544]
 [  432  1116   187 ... 12306 12304     0]
 [ 7837   297   102 ...  9843  9836     0]
 ...
 [ 3177 18755 17703 ...  9144  9138     0]
 [ 3177 18755 17703 ...  9144  9138     0]
 [19089 14604 18737 ... 10806 10805     0]]


In [ ]:
similarity_genre.shape

(19090, 19090)

In [ ]:
def recommend_movie_list(x, title, top= 30):
# 특정 영화 정보 뽑아내기
  target_movie_index = x[x['title'] == title].index.values
# 타겟 영화와 비슷한 코사인 유사도 값
  sim_index = similarity_genre[target_movie_index, :top].reshape(-1)
# 본인 제외
  sim_index = sim_index[sim_index!=target_movie_index]
# 추천 결과 새로운 df 생성 , 컨텐츠 레이팅으로 정렬
  result = x.iloc[sim_index].sort_values('contentRating',ascending=False)[:10]

  return result

In [ ]:
recommend_movie_list(x, title = '게게게의 키타로')

,contentId,title,genre,contentRating
15293,44893,드림헌터 레무,Sci-Fi Fantasy Animation,9.0
17690,74064,고백부부,Comedy Sci-Fi Fantasy,8.2
13505,60998,Every Witch Way,Comedy Sci-Fi Fantasy,8.2
11008,92779,외성여생시소칠 : 내 여자친구는 외계인,Comedy Sci-Fi Fantasy,8.2
15439,61174,비와 퍼피캣,Sci-Fi Fantasy Animation,8.1
12286,326,Red Dwarf,Comedy Sci-Fi Fantasy,8.1
11795,86248,업로드,Comedy Sci-Fi Fantasy,8.1
12486,78102,슈타인즈 게이트 제로,Sci-Fi Fantasy Animation,8.1
18298,87971,애상북두성남우,Comedy Sci-Fi Fantasy,8.0
18466,136738,울프 라이크 미,Comedy Sci-Fi Fantasy,8.0
